In [ ]:
# Install dependencies - não é necessário correr
import sys
!{sys.executable} -m pip install hdfs

In [13]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType, DoubleType
from pyspark.sql.functions import when, col, concat, lit, substring, avg, sum

client = InsecureClient("http://hdfs-nn:9870", user="anonymous")

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/fisico/gold/'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [14]:
# apagar tudo do hdfs

to_path = "/projeto/fisico/gold/stats/"

client.delete(to_path,True)

True

In [15]:
# read data from the silver tables
df = spark.table("silver.stats")

In [16]:
df.printSchema()
df.count()

root
 |-- Year: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tm: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- G: integer (nullable = true)
 |-- GS: integer (nullable = true)
 |-- Wins: integer (nullable = true)
 |-- Losses: integer (nullable = true)
 |-- WP: double (nullable = true)
 |-- MP: integer (nullable = true)
 |-- PER: double (nullable = true)
 |-- TSP: double (nullable = true)
 |-- 3PAr: double (nullable = true)
 |-- FTr: double (nullable = true)
 |-- ORBP: double (nullable = true)
 |-- DRBP: double (nullable = true)
 |-- TRBP: double (nullable = true)
 |-- ASTP: double (nullable = true)
 |-- STLP: double (nullable = true)
 |-- BLKP: double (nullable = true)
 |-- TOVP: double (nullable = true)
 |-- USGP: double (nullable = true)
 |-- OWS: double (nullable = true)
 |-- DWS: double (nullable = true)
 |-- WS: double (nullable = true)
 |-- WS48: double (nullable = tru

25475

In [ ]:
gd_df.createOrReplaceTempView("gd_df")

spark.sql(
"""
SELECT 3P from gd_df WHERE Year=1980
"""
).show()

In [17]:
# apagar todos os dados anteriores à introdução dos triplos
gd_df = df.where("Year > 1979")

In [18]:
gd_df = gd_df.withColumn("Year", gd_df["Year"].cast(IntegerType()))

In [19]:
gd_df.printSchema()
gd_df.count()

root
 |-- Year: integer (nullable = true)
 |-- Player: string (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tm: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- G: integer (nullable = true)
 |-- GS: integer (nullable = true)
 |-- Wins: integer (nullable = true)
 |-- Losses: integer (nullable = true)
 |-- WP: double (nullable = true)
 |-- MP: integer (nullable = true)
 |-- PER: double (nullable = true)
 |-- TSP: double (nullable = true)
 |-- 3PAr: double (nullable = true)
 |-- FTr: double (nullable = true)
 |-- ORBP: double (nullable = true)
 |-- DRBP: double (nullable = true)
 |-- TRBP: double (nullable = true)
 |-- ASTP: double (nullable = true)
 |-- STLP: double (nullable = true)
 |-- BLKP: double (nullable = true)
 |-- TOVP: double (nullable = true)
 |-- USGP: double (nullable = true)
 |-- OWS: double (nullable = true)
 |-- DWS: double (nullable = true)
 |-- WS: double (nullable = true)
 |-- WS48: double (nullable = tr

19778

In [ ]:
# apagar coluna da idade pois já existe no gd_physical
# gd_df = gd_df.select("Year","Player","Pos","Tm","Team","G","GS","MP","PER","TSP","3PAr","FTr","ORBP","DRBP","TRBP","ASTP","STLP","BLKP","TOVP","USGP","OWS","DWS","WS","WS48","OBPM","DBPM","BPM","VORP","FG","FGA","FGP","3P","3PA","3PP","2P","2PA","2PP","eFGP","FT","FTA","FTP","ORB","DRB","TRB","AST","STL","BLK","TOV","PF","PTS")

In [22]:
# write to delta table
gd_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/projeto/fisico/gold/stats/")

In [ ]:
# check the results in the table
spark.table("gold.stats").show()
spark.table("gold.stats").count()